Some setup notes and Help - 
Patrick Connelly
pconnell89@gmail.com

Getting Fair Face Up-and-running

To get all of the data to start some work, I began by downloading some files and storing them in a folder structure.

Spreadsheet/CSV is located here: https://github.com/Raschka-research-group/coral-cnn/tree/master/datasets.  I believe I was using the afad_test csv.

The images themselves for AFAD (Asian Face Age Dataset) are tarballs located in a separate repo on github: https://github.com/John-niu-07/tarball.  To decompress / extract them, you can use 7-zip on windows, or tar xvf filename_here on linux or mac.

Some other dependencies - 

    1) Install windows subsystem for linux (WSL)

    2) Simple method for Windows - install miniconda on WSL

        - enter WSL shell
    
        - wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    
        - bash Miniconda3-latest-Linux-x86_64.sh
    
        - rm Miniconda3-latest-Linux-x86_64.sh
    
        - conda init
    
        - conda activate

    3) Create a python env within conda (https://learn.microsoft.com/en-us/windows/ai/directml/gpu-pytorch-windows)

        - conda create --name directml python=3.8 ipython (or whatever you want) 

        - conda activate directml (to enter the environment)

        - Within the environment - 

            a) Install Python 3.8

            b) Pip install torch - https://pytorch.org/get-started/locally/

            c) Pip torch_directml, numpy, pandas, dlib, deepface

Overall purpose of the code in this file is to identify locally stored and unzipped data, and use that to filter down the spreadsheet of data from a source dataset. 
The output provides you with a csv called "img_paths" which can be fed to deepFaceIteration.py or to predict.py as a parameter for iteration.



FairFace and DeepFace Master Script
Author: Patrick Connelly
Purpose:

Design scripts that provide ease of running both the FairFace and DeepFace models within a Python 3.8 environment

Non-Functional Facial Detection Backends (DeepFace):

* ssd
* dlib
* mediapipe
* yolov8
* yunet

This leaves opencv, ssd, mtcnn, and retinaface as available facial detection backends.

ssd requires resizing and opencv commonly fails to detect faces with parameter enforce_detection = True.





Downloading and Setting Up FairFace

* Clone the repo:
* install dependencies (PyTorch, pandas, dlib)
* 

In [1]:
#fairface modified code.

from __future__ import print_function, division
import warnings
warnings.filterwarnings("ignore")
import os.path
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch_directml
import dlib
import os
import argparse

def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

def detect_face(image_paths,  SAVE_DETECTED_AT, default_max_size=800,size = 300, padding = 0.25):
    cnn_face_detector = dlib.cnn_face_detection_model_v1('dlib_models/mmod_human_face_detector.dat')
    sp = dlib.shape_predictor('dlib_models/shape_predictor_5_face_landmarks.dat')
    base = 2000  # largest width and height
    for index, image_path in enumerate(image_paths):
        if index % 1000 == 0:
            print('---%d/%d---' %(index, len(image_paths)))
        img = dlib.load_rgb_image(image_path)

        old_height, old_width, _ = img.shape

        if old_width > old_height:
            new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
        else:
            new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
        img = dlib.resize_image(img, rows=new_height, cols=new_width)

        dets = cnn_face_detector(img, 1)
        num_faces = len(dets)
        if num_faces == 0:
            print("Sorry, there were no faces found in '{}'".format(image_path))
            continue
        # Find the 5 face landmarks we need to do the alignment.
        faces = dlib.full_object_detections()
        for detection in dets:
            rect = detection.rect
            faces.append(sp(img, rect))
        images = dlib.get_face_chips(img, faces, size=size, padding = padding)
        for idx, image in enumerate(images):
            img_name = image_path.split("/")[-1]
            path_sp = img_name.split(".")
            face_name = os.path.join(SAVE_DETECTED_AT,  path_sp[0] + "_" + "face" + str(idx) + "." + path_sp[-1])
            dlib.save_image(image, face_name)

def detect_face_single(image_path, save=False, SAVE_DETECTED_AT='', default_max_size=800, size=300, padding-0.25):
    cnn_face_detector = dlib.cnn_face_detection_model_v1('dlib_models/mmod_human_face_detector.dat')
    sp = dlib.shape_predictor('dlib_models/shape_predictor_5_face_landmarks.dat')
    base = 2000
    img = dlib.load_rgb_image(image_path)
    old_height,old_width, _ = img.shape
    if old_width > old_height:
        new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
    else:
        new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
    img = dlib.resize_image(img, rows=new_height, cols=new_width)
    dets = cnn_face_detector(img, 1)
    num_faces = len(dets)
    if num_faces == 0:
        print("Sorry, there were no faces found in '{}'".format(image_path))
    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()

    for detection in dets:
        rect = detection.rect
        faces.append(sp(img, rect))

    images = dlib.get_face_chips(img, faces, size=size, padding = padding)
    # for idx, image in enumerate(images):
    #     img_name = image_path.split("/")[-1]
    #     path_sp = img_name.split(".")
    #     face_name = os.path.join(SAVE_DETECTED_AT,  path_sp[0] + "_" + "face" + str(idx) + "." + path_sp[-1])
    #     dlib.save_image(image, face_name)

    return images


def save_preprocessed(SAVE_DETECTED_AT,images):
    for idx, image in enumerate(images):
        img_name = image_path.split("/")[-1]
        path_sp = img_name.split(".")
        face_name = os.path.join(SAVE_DETECTED_AT,  path_sp[0] + "_" + "face" + str(idx) + "." + path_sp[-1])
        dlib.save_image(image, face_name)


def get_os_torch_device():
    dev = None
    if torch.cuda.is_available():
        dev = torch.device('cuda:0')
    else:
        dev = torch.device(torch_directml.device())
    
    return dev

device = get_os_torch_device()
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_featurese,18)

def pred_age_gen_race_single(save_prediction_at,img_path,enforce_detection=False):
    if enforce_detection == True:
        #assume single face
        image = detect_face_single(img_path)[0]
    else:
        image = dlib.load_rgb_image(img_path)

    


    pass

def predidct_age_gender_race(save_prediction_at, imgs_path = 'cropped_faces/'):
    img_names = [os.path.join(imgs_path, x) for x in os.listdir(imgs_path)]
    #original code line
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #added torch_directml to allow evaluation to take place on GPU with model loaded to CPU. It's weird but works on AMD.
    device = torch.device(torch_directml.device())
    model_fair_7 = torchvision.models.resnet34(pretrained=True)
    model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
    #original code line
    #model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt'))
    #changed this to match the filename of the online downloadable dataseet and to use CPU because I have AMD graphics, and torch.load doesn't like directml
    model_fair_7.load_state_dict(torch.load('fair_face_models/res34_fair_align_multi_7_20190809.pt',map_location=torch.device('cpu')))
    model_fair_7 = model_fair_7.to(device)
    model_fair_7.eval()

    model_fair_4 = torchvision.models.resnet34(pretrained=True)
    model_fair_4.fc = nn.Linear(model_fair_4.fc.in_features, 18)
    #original code line
    #model_fair_4.load_state_dict(torch.load('fair_face_models/fairface_alldata_4race_20191111.pt'))
    #changed this to match the filename of the online downloadable dataseet and to use CPU because I have AMD graphics, and torch.load doesn't like directml
    model_fair_4.load_state_dict(torch.load('fair_face_models/res34_fair_align_multi_4_20190809.pt',map_location=torch.device('cpu')))
    model_fair_4 = model_fair_4.to(device)
    model_fair_4.eval()

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # img pth of face images
    face_names = []
    # list within a list. Each sublist contains scores for all races. Take max for predicted race
    race_scores_fair = []
    gender_scores_fair = []
    age_scores_fair = []
    race_preds_fair = []
    gender_preds_fair = []
    age_preds_fair = []
    race_scores_fair_4 = []
    race_preds_fair_4 = []

    for index, img_name in enumerate(img_names):
        if index % 1000 == 0:
            print("Predicting... {}/{}".format(index, len(img_names)))

        face_names.append(img_name)
        image = dlib.load_rgb_image(img_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device)

        # fair
        outputs = model_fair_7(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        race_outputs = outputs[:7]
        gender_outputs = outputs[7:9]
        age_outputs = outputs[9:18]

        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        age_score = np.exp(age_outputs) / np.sum(np.exp(age_outputs))

        race_pred = np.argmax(race_score)
        gender_pred = np.argmax(gender_score)
        age_pred = np.argmax(age_score)

        race_scores_fair.append(race_score)
        gender_scores_fair.append(gender_score)
        age_scores_fair.append(age_score)

        race_preds_fair.append(race_pred)
        gender_preds_fair.append(gender_pred)
        age_preds_fair.append(age_pred)

        # fair 4 class
        outputs = model_fair_4(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        race_outputs = outputs[:4]
        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        race_pred = np.argmax(race_score)

        race_scores_fair_4.append(race_score)
        race_preds_fair_4.append(race_pred)

    result = pd.DataFrame([face_names,
                           race_preds_fair,
                           race_preds_fair_4,
                           gender_preds_fair,
                           age_preds_fair,
                           race_scores_fair, race_scores_fair_4,
                           gender_scores_fair,
                           age_scores_fair, ]).T
    result.columns = ['face_name_align',
                      'race_preds_fair',
                      'race_preds_fair_4',
                      'gender_preds_fair',
                      'age_preds_fair',
                      'race_scores_fair',
                      'race_scores_fair_4',
                      'gender_scores_fair',
                      'age_scores_fair']
    result.loc[result['race_preds_fair'] == 0, 'race'] = 'White'
    result.loc[result['race_preds_fair'] == 1, 'race'] = 'Black'
    result.loc[result['race_preds_fair'] == 2, 'race'] = 'Latino_Hispanic'
    result.loc[result['race_preds_fair'] == 3, 'race'] = 'East Asian'
    result.loc[result['race_preds_fair'] == 4, 'race'] = 'Southeast Asian'
    result.loc[result['race_preds_fair'] == 5, 'race'] = 'Indian'
    result.loc[result['race_preds_fair'] == 6, 'race'] = 'Middle Eastern'

    # race fair 4

    result.loc[result['race_preds_fair_4'] == 0, 'race4'] = 'White'
    result.loc[result['race_preds_fair_4'] == 1, 'race4'] = 'Black'
    result.loc[result['race_preds_fair_4'] == 2, 'race4'] = 'Asian'
    result.loc[result['race_preds_fair_4'] == 3, 'race4'] = 'Indian'

    # gender
    result.loc[result['gender_preds_fair'] == 0, 'gender'] = 'Male'
    result.loc[result['gender_preds_fair'] == 1, 'gender'] = 'Female'

    # age
    result.loc[result['age_preds_fair'] == 0, 'age'] = '0-2'
    result.loc[result['age_preds_fair'] == 1, 'age'] = '3-9'
    result.loc[result['age_preds_fair'] == 2, 'age'] = '10-19'
    result.loc[result['age_preds_fair'] == 3, 'age'] = '20-29'
    result.loc[result['age_preds_fair'] == 4, 'age'] = '30-39'
    result.loc[result['age_preds_fair'] == 5, 'age'] = '40-49'
    result.loc[result['age_preds_fair'] == 6, 'age'] = '50-59'
    result.loc[result['age_preds_fair'] == 7, 'age'] = '60-69'
    result.loc[result['age_preds_fair'] == 8, 'age'] = '70+'

    result[['face_name_align',
            'race', 'race4',
            'gender', 'age',
            'race_scores_fair', 'race_scores_fair_4',
            'gender_scores_fair', 'age_scores_fair']].to_csv(save_prediction_at, index=False)

    print("saved results at ", save_prediction_at)


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def detect_ind_face(image_path,  SAVE_DETECTED_AT):
    frame=pd.DataFrame({'img_path':[image_path]})
    image_paths = frame['img_path']
    detect_face(image_paths,  SAVE_DETECTED_AT)

def analyze_ind_face(save_prediction_at, imgs_path = 'cropped_faces/'):
    pass

# if __name__ == "__main__":
#     #Please create a csv with one column 'img_path', contains the full paths of all images to be analyzed.
#     #Also please change working directory to this file.
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--csv', dest='input_csv', action='store',
#                         help='csv file of image path where col name for image path is "img_path')
#     dlib.DLIB_USE_CUDA = False #True
#     print("using CUDA?: %s" % dlib.DLIB_USE_CUDA)
#     args = parser.parse_args()
#     SAVE_DETECTED_AT = "detected_faces"
#     ensure_dir(SAVE_DETECTED_AT)
#     imgs = pd.read_csv(args.input_csv)['img_path']
#     detect_face(imgs, SAVE_DETECTED_AT)
#     print("detected faces are saved at ", SAVE_DETECTED_AT)
#     #Please change test_outputs.csv to actual name of output csv. 
#     predidct_age_gender_race("test_outputs.csv", SAVE_DETECTED_AT)


In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
afad_test = pd.read_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_test.csv')


Below, I pull in the CSV from the Asian Face Dataset.  I downloaded the CSV and one chunk of the images locally.

This first block below is just to test and verify that I was able to bring in the information.

In [3]:
afad_test = pd.read_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_test.csv')
afad_test.head(7)

,file,path,age,gender
0,406208-0.jpg,39/112/406208-0.jpg,24,female
1,470804-0.jpg,39/112/470804-0.jpg,24,female
2,411657-2.jpg,39/112/411657-2.jpg,24,female
3,471528-2.jpg,39/112/471528-2.jpg,24,female
4,398608-1.jpg,39/112/398608-1.jpg,24,female
5,470572-0.jpg,39/112/470572-0.jpg,24,female
6,401238-0.jpg,39/112/401238-0.jpg,24,female


Spreadsheet/CSVs were located here on the web: https://github.com/Raschka-research-group/coral-cnn/tree/master/datasets 

The images themselves are tarballs located in a separate repo on github: https://github.com/John-niu-07/tarball.

In terms of this dataset - the images are "chunked" into tarball/zip files in GitHub for the AFAD dataset.  The spreadsheet/csv, however, has a listing of ALL file paths for all files in the dataset.  In order to get to a point where I could only iterate over the target files, I had to do a directory walk to identify only .jpg files, searching recursively through a local folder.  

This folder ahd the repo / files from a git clone of FairFace, models downloaded separately from Google Drive (not in the GitHub repo directly), the chunk of images from the AFAD dataface extracted into two subfolders, and the full spreadsheet of all items in the AFAD dataset.

So I did the directory walk to get a list of all JPGs.  I left-joined the source spreadsheet (left table) to the present files (right table), and then filtered where the right table was blank/NA, so that the result was a table only of the available files for iteration, and then wrote that out to a csv.

I used this to build my input for my deepFaceIteration.py script and the modified version of predict.py from the FairFace model.

I didn't set this up with functions or parameters (i.e. tell me where you wanna save the file).  

In [4]:
PATH = "C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full"
EXT = "*.jpg"

# get a list of all filenames recursively from the current directory in which this script runs
# and down into any subfolders as well

all_jpg = [file 
           for path, subdir, files in os.walk(PATH)
           for file in glob(os.path.join(path,EXT))]

#can be used to verify how many you identified in the local folder(s)
#len(all_jpg)
#all_jpg[0]

present_files = pd.DataFrame({'jpgs': all_jpg})
present_files['file']=(present_files['jpgs'].str.split('\\')).str[-1]
present_files

#do a left join between the files we know could exist (left) and the files we found (right)
#filter the list down only to items where there was a match as well (maybe inner join is smarter)
merged=afad_test.merge(present_files,on='file',how='left')
merged = merged[merged['jpgs'].notna()]

#trim and generate the output file!
merged['jpgs'] = merged['jpgs'].str.replace('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full','.')
merged['img_path'] = merged['jpgs']
merged=merged.drop(labels=['jpgs','path','file','age','gender'],axis=1)

merged.to_csv('C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_loc_clean.csv',index=False)

len(merged)

491